In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
bike_infra = "Bike_Network.csv"
bike_infra_df = pd.read_csv(bike_infra)
bike_infra_df.head()

,OBJECTID,STREETNAME,TYPE
0,1,BARTRAM AVE,Conventional
1,2,BARTRAM AVE,Conventional
2,3,BARTRAM AVE,Conventional
3,4,ISLAND AVE,Paint Buffered
4,5,BARTRAM AVE,Conventional


In [3]:
acc_data = "accident_data_philly.csv"
acc_data_df = pd.read_csv(acc_data)
acc_data_df.head()

,DATE,Time,STREET NAME,VEHICLES,TOTAL PEOPLE,PEDESTRIANS,BICYCLISTS,INJURIES,MAJOR INJURIES,FATALITIES
0,11-Jan,3:36 a.m.,BENSON ST & FRANKFORD AV,2,3,1,0,0,0,1
1,11-Jan,21:00 p.m.,CONSHOHOCKEN AV,1,2,1,0,1,1,0
2,11-Jan,17:45 p.m.,WOODLAND AV,1,2,1,0,0,0,1
3,11-Jan,3:10 a.m.,GLENWOOD AV,1,2,1,0,1,1,0
4,11-Feb,16:37 p.m.,N FRONT ST,1,2,1,0,0,0,1


In [4]:
# Create a filtered dataframe from specific columns of the bike infrastrucuture dataframe
# with values street names, street code, and type of infrastructure 

bike_infra_cols = bike_infra_df[["OBJECTID","STREETNAME", "TYPE"]].copy()
bike_infra_cols = bike_infra_cols.rename(columns={"TYPE": "lane_type", 
                                                 "STREETNAME": "street_name_lane", "OBJECTID":"objectid"})


In [5]:
#length of df 
len(bike_infra_cols)

5101

In [6]:
#length after dropping duplicates based on streetname and lane type
bike_infra_cols_cleaned = bike_infra_cols.drop_duplicates(subset=['street_name_lane','lane_type'])
len(bike_infra_cols_cleaned)

431

In [7]:
#cleaning data and deleting extra space in between street name
bike_list = []
for row in bike_infra_cols_cleaned["street_name_lane"]:
    row = row.replace('  ',' ')
    bike_list.append(row)

In [8]:
#putting cleaned data column back into table
bike_infra_cols_cleaned['street_name_lane'] = bike_list
bike_infra_cols_cleaned

C:\Users\ramya\AppData\Local\Temp\ipykernel_30656\3706265764.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bike_infra_cols_cleaned['street_name_lane'] = bike_list


,objectid,street_name_lane,lane_type
0,1,BARTRAM AVE,Conventional
3,4,ISLAND AVE,Paint Buffered
5,6,S 84TH ST,Conventional
8,9,MARIO LANZA BLVD,Conventional
9,10,LINDBERGH BLVD,Paint Buffered
...,...,...,...
5041,5042,JAMISON AVE,Conventional
5044,5045,TOMLINSON RD,Conventional
5056,5057,SOUTHAMPTON RD,Conventional
5060,5061,TOWNSEND RD,Conventional


In [9]:
# Create a filtered dataframe from specific columns of the bike accident dataframe
# with values street names, street code, and type of infrastructure 
acc_data_cols = acc_data_df[["STREET NAME", "BICYCLISTS", "INJURIES", "MAJOR INJURIES"]].copy()
acc_data_cols = acc_data_cols.rename(columns={"STREET NAME": "street_name", 
                                                 "MAJOR INJURIES": "major_injuries", 
                                                 "BICYCLISTS": "bicyclists", "INJURIES": "injuries" })

#splitting street names, so it doesn't show intersections
for row in acc_data_cols.street_name:
    row.split(" &")
acc_data_cols

,street_name,bicyclists,injuries,major_injuries
0,BENSON ST & FRANKFORD AV,0,0,0
1,CONSHOHOCKEN AV,0,1,1
2,WOODLAND AV,0,0,0
3,GLENWOOD AV,0,1,1
4,N FRONT ST,0,0,0
...,...,...,...,...
868,CLEARFIELD ST & KENSINGTON AV & G ST,1,1,1
869,ANDREWS AV,0,1,1
870,BROAD ST & CHESTNUT ST,0,1,1
871,13TH ST & PINE ST,1,1,1


In [10]:
#putting cleaned data back into table
acc_data_cols["street_name"] = acc_data_cols["street_name"].str.split(" ")
acc_data_cols["street_name"]

0                     [BENSON, ST, &, FRANKFORD, AV]
1                                 [CONSHOHOCKEN, AV]
2                                     [WOODLAND, AV]
3                                     [GLENWOOD, AV]
4                                     [N, FRONT, ST]
                           ...                      
868    [CLEARFIELD, ST, &, KENSINGTON, AV, &, G, ST]
869                                    [ANDREWS, AV]
870                     [BROAD, ST, &, CHESTNUT, ST]
871                          [13TH, ST, &, PINE, ST]
872                       [LAPSLEY, RD, &, CITY, AV]
Name: street_name, Length: 873, dtype: object

In [11]:
#test 
strings = acc_data_cols["street_name"][0][:2]
a = ','.join(strings)
b = a.replace(',', ' ')
b

'BENSON ST'

In [12]:
#replacing AV with AVE, after separating street name into lists, deleting everything after second item, so only first street
new_list = []
for row in acc_data_cols["street_name"]:
    row = ','.join(row[:2]).replace(',', ' ')
    row = row.replace('AV','AVE')
    new_list.append(row)

In [13]:
#putting cleaned data back into table, into new column
acc_data_cols['street_name_clean'] = new_list
acc_data_cols.head(30)

,street_name,bicyclists,injuries,major_injuries,street_name_clean
0,"[BENSON, ST, &, FRANKFORD, AV]",0,0,0,BENSON ST
1,"[CONSHOHOCKEN, AV]",0,1,1,CONSHOHOCKEN AVE
2,"[WOODLAND, AV]",0,0,0,WOODLAND AVE
3,"[GLENWOOD, AV]",0,1,1,GLENWOOD AVE
4,"[N, FRONT, ST]",0,0,0,N FRONT
5,"[CAMBRIA, ST, &, FOURTH, ST]",0,1,1,CAMBRIA ST
6,"[ARCH, ST]",0,1,1,ARCH ST
7,"[TWELFTH, ST]",0,1,1,TWELFTH ST
8,"[PASSYUNK, AV, &, RITNER, ST]",0,1,1,PASSYUNK AVE
9,"[TABOR, RD]",0,1,1,TABOR RD


In [14]:
#creating connection with PgAdmin
connection_string = "postgres:postgres@localhost:5432/bike_acc_and_infra_db"
engine = create_engine(f'postgresql://{connection_string}')

In [15]:
#getting table names
engine.table_names()

C:\Users\ramya\AppData\Local\Temp\ipykernel_30656\4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['infrastructure', 'accidents']

In [22]:
#loading data into tables
bike_infra_cols_cleaned.to_sql(name='infrastructure', con=engine, if_exists='append', index=False)

431

In [23]:
#loading data into tables
acc_data_cols.to_sql(name='accidents', con=engine, if_exists='append', index=False)

873